# Extracción de datos Datasets Google Maps y YELP


### Importaciones


In [5]:
import pandas as pd
import numpy as np
import json
import datetime
import os
import pyarrow.parquet as pq
from datetime import datetime

### Utiles


In [6]:
state_abreviations = [
    "AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA",
    "KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ",
    "NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT",
    "VA","WA","WV","WI","WY",
]

state_dictionary = {
    "AL": "Alabama","AK": "Alaska","AZ": "Arizona","AR": "Arkansas",
    "CA": "California","CO": "Colorado","CT": "Connecticut","DE": "Delaware",
    "FL": "Florida","GA": "Georgia","HI": "Hawaii","ID": "Idaho",
    "IL": "Illinois","IN": "Indiana","IA": "Iowa","KS": "Kansas",
    "KY": "Kentucky","LA": "Louisiana","ME": "Maine","MD": "Maryland",
    "MA": "Massachusetts","MI": "Michigan","MN": "Minnesota",
    "MS": "Mississippi","MO": "Missouri","MT": "Montana","NE": "Nebraska",
    "NV": "Nevada","NH": "New Hampshire","NJ": "New Jersey","NM": "New Mexico",
    "NY": "New York","NC": "North Carolina","ND": "North Dakota","OH": "Ohio",
    "OK": "Oklahoma","OR": "Oregon","PA": "Pennsylvania","RI": "Rhode Island",
    "SC": "South Carolina","SD": "South Dakota","TN": "Tennessee",
    "TX": "Texas","UT": "Utah","VT": "Vermont","VA": "Virginia",
    "WA": "Washington","WV": "West Virginia","WI": "Wisconsin","WY": "Wyoming",
}

In [7]:
generated_dir = "Generated"

os.mkdir(generated_dir)
os.chdir(generated_dir)
os.mkdir("Google")
os.mkdir("Yelp")
os.chdir("../")

os.getcwd()

FileExistsError: [WinError 183] No se puede crear un archivo que ya existe: 'Generated'

# 1. Extracción con los datasets de Google Maps


## 1.1 Metada de Sitios


Recorremos todo el directorio, lo hacemos archivo por archivo y línea por línea ya que no se puede abrir directamente los archivos por su dimensión y porque no están en formato de array, sino están constituidos en un registro por lìnea.
Durante la lectura filtramos los que incluyan <code>Restaurant</code> en el campo de categoría, para alivianar el dataset final.


In [ ]:
# Tiempo de Demora Medio: 51 segundos.
lineas_json = []

# Son 11 archivos con un ordinal, del 1 al 11
for i in range(1, 12):
    path = f"Datasets/Google Maps/metadata-sitios/{i}.json"
    with open(path, "r") as file:
        for l in file:
            try:
                linea_j = json.loads(l)
                if "restaurant" in " ".join(linea_j["category"]).lower():
                    lineas_json.append(linea_j)
            except:
                pass

df = pd.DataFrame(lineas_json)

df.head(3)

In [ ]:
df.loc[5, "address"]

Exportamos a formato Parquet


In [ ]:
df.to_parquet(r"Generated\Google\metada_sitios.parquet")

Tamaño Directorio <code>metadata-sitios</code>: 2.76 Gb

Tamaño Archivo <code>metada_sitios.parquet</code>: 60.43 Mb


In [ ]:
df.info()

Dimensiones: 212.014 filas x 15 Columnas


## 1.1.1 Obtención de información de Estados


En base al campo <code>Address</code> obtenemos el estado donde se encuentra el negocio. Nos servirá para luego seleccionar los estados con más restaurantes.


In [ ]:
def get_state_ab(st):
    try:
        state = st.split(", ")[-1].split(" ")[0]
        if state in state_abreviations:
            return state
        else:
            return np.nan
    except:
        return np.nan


df["state_ab"] = df["address"].apply(get_state_ab)

De esta manera conseguimos el top 5 de los estados con más restaurantes


In [ ]:
top_5 = df["state_ab"].value_counts().head(5).index.to_list()

In [ ]:
top_5

Completamos el campo estado que es más descriptivo


In [ ]:
df["us_state"] = df["state_ab"].map(state_dictionary)

In [ ]:
df["us_state"].head(5)

Obtenemos un arreglo de URLs de los archivos correspondientes para cada estado del top 5, con el fin de extraer los datos en un bucle.


In [ ]:
top_5_url = [
    f"Datasets/Google Maps/reviews-estados/review-{state_dictionary[i].replace(' ', '_')}/"
    for i in top_5
]



top_5_url

Creamos un diccionario con la cantidad de archivos por cada directorio de estados, con el fin de utilizarlo en un bucle en la extracción de datos.


In [ ]:
cantidad_archivos = {}

for i in top_5_url:
    for j in os.walk(i):
        cantidad_archivos[i] = len(j[2])

cantidad_archivos

Comprobación


In [ ]:
for i in os.walk("Datasets/Google Maps/reviews-estados/review-Pennsylvania"):
    print(len(i[2]))

In [ ]:
cantidad_archivos["Datasets/Google Maps/reviews-estados/review-Pennsylvania/"]

## 1.2 Reviews Estados


Ya con los estados elegidos estamos en condiciones de ingestar los datos de las carpetas correspondientes dentro del directorio <code>reviews-estados</code>.
Es información masiva lo que genera un archivo de grandes dimensiones, sin embargo previamente filtramos por el parámetro de año <code>2017-2019</code> valiéndonos del campo <code>time</code>, que tiene es un <code>timestamp</code>, pero con 3 digitos más que el usado por <code>datetime</code> de Python. Le agregamos el campo <code>Estado</code> que es más descriptivo.


In [ ]:
### Demora 7 minutos y 40 segundos, 11 minutos, varía

lineas_json_revs_google = []

for i in top_5_url:
    count = 0
    for c in range(1, cantidad_archivos[i] + 1):
        with open(str(i) + str(c) + ".json", "r", encoding="utf-8") as f:
            for s in f:
                linea = json.loads(s)
                linea["anio"] = datetime.datetime.fromtimestamp(
                    linea["time"] / 1000
                ).year
                linea["estado"] = i.split("-")[-1][:-1]

                if linea["anio"] in [2017, 2018, 2019]:
                    lineas_json_revs_google.append(linea)

df_revs_google = pd.DataFrame(lineas_json_revs_google)

df_revs_google.head(3)

In [ ]:
merge_site_reviews = pd.merge(df_revs_google, df, left_on="gmap_id", right_on="gmap_id")

In [ ]:
merge_site_reviews

In [ ]:
merge_site_reviews.to_parquet(r"Generated\Google\merge_site_reviews.parquet")

In [ ]:
df_revs_google.to_parquet(r"Generated\Google\reviews-estados.parquet")

Tamaño archivo: 760 Mb

Tamaño dataset: 24.3 Gb


In [ ]:
df_revs_google.info()

Tamaño 8.339.179 filas x 10 Columnas.


# 2. Extracción de los Dataset de YELP


### 2.1 Business


Contiene los datos de las entidades negocios de Yelp, a un primer vistazo tiene las columnas duplicadas, por lo que hay que hacer un recorte, ya que la segunda mitad tiene datos vacíos en su inmensa mayoría.


In [ ]:
url_business = r"Datasets\Yelp\business.pkl"

df_business = pd.read_pickle(url_business)

df_business = df_business.iloc[:, :-14]

In [ ]:
df_business.sample(3)

In [ ]:
df_business.shape

Luego con la ayuda del campo <code>state</code> filtramos los negocios que se encuentran en los estados seleccionados en nuestro análisis.


In [ ]:
df_business = df_business[df_business.state.isin(top_5)]

In [ ]:
df_business.info()

Seguimos filtrando a través del campo <code>categories</code>, para obtener los negocios que son restaurantes.


In [ ]:
def is_restaurant(st):
    try:
        test = "".join(st).lower()
        return "restaurant" in test
    except:
        return False


df_business = df_business[df_business["categories"].apply(is_restaurant)]

In [ ]:
df_business.info()

In [ ]:
df_business.to_parquet(r"Generated\Yelp\bussines.parquet")

### 2.2 Checkin


In [ ]:
lineas_json = []
path_checkin = r"Datasets\Yelp\checkin.json"
with open(path_checkin, "r", encoding="utf-8") as file:
    for l in file:
        try:
            linea_j = json.loads(l)
            anio = linea_j["date"][:4]
            # if 'restaurant' in " ".join(linea_j['category']).lower():
            if anio in ["2017", "2018", "2019"]:
                lineas_json.append(linea_j)
        except:
            pass

df_checkin = pd.DataFrame(lineas_json)

In [ ]:
df_checkin

In [ ]:
merge_business_checkin = pd.merge(
    df_business, df_checkin, left_on="business_id", right_on="business_id"
)

In [ ]:
merge_business_checkin.info()

In [ ]:
df_checkin.to_parquet(r"Generated\Yelp\checkin.parquet")
merge_business_checkin.to_parquet(r"Generated\YELP\business_checkin.parquet")

### 2.3 Tips


Realizamos la extracción de los datos y filtramos por año según nuestro análisis.


In [ ]:
lineas_json = []
path_tip = r"Datasets\Yelp\tip.json"
with open(path_tip, "r", encoding="utf-8") as file:
    for l in file:
        try:
            linea_j = json.loads(l)
            anio = linea_j["date"][:4]
            if anio in ["2017", "2018", "2019"]:
                lineas_json.append(linea_j)
        except:
            pass

df_tip = pd.DataFrame(lineas_json)

In [ ]:
df_tip.sample(5)

In [ ]:
df_tip.to_parquet(r"Generated\Yelp\tip.parquet")

Unimos el el dataframe de tips con el de negocios


In [ ]:
tips_merged = pd.merge(
    df_tip, df_business, left_on="business_id", right_on="business_id"
)

In [ ]:
tips_merged.sort_values("business_id").head(3)

In [ ]:
tips_merged.to_parquet(r"Generated\Yelp\business_tip.parquet")

### 2.4 Review


In [ ]:
df_reviews_url = r"Datasets\Yelp\review.json"

Usamos el mismo método de linea por linea, y en el proceso filtramos por año y por las reseñas que han sido votadas como útiles.


In [ ]:
# 44 segundos

lineas_json_review = []

with open(df_reviews_url, "r", encoding="utf-8") as f:
    count = 0
    for i in f:
        linea = json.loads(i)
        anio = linea["date"][:4]
        if anio in ["2017", "2018", "2019"] and linea["useful"] == 1:
            lineas_json_review.append(linea)


df_reviews = pd.DataFrame(lineas_json_review)

In [ ]:
df_reviews.sample(10)

In [ ]:
df_reviews.info()

Aligeramos el dataset con unos downgrades de tipo de variables.


In [ ]:
df_reviews["funny"] = df_reviews["funny"].astype("int8")
df_reviews["stars"] = df_reviews["stars"].astype("int8")
df_reviews["cool"] = df_reviews["cool"].astype("int8")

df_reviews.drop("useful", axis=1, inplace=True, errors="ignore")

In [ ]:
df_reviews.to_parquet(r"Generated\Yelp\review.parquet")

### 2.4 Users Yelp


In [ ]:
parquet_file = pq.ParquetFile(r"Datasets\Yelp\user.parquet")

arr_df = []

for batch in parquet_file.iter_batches():
    # count = count +1
    batch_df = batch.to_pandas()
    batch_df["elite"] = batch_df["elite"].apply(lambda x: x.split(","))
    batch_df["elite_len"] = batch_df["elite"].apply(lambda x: len(x))
    batch_df = batch_df.query("elite_len > 1")
    arr_df.append(batch_df)

df_users = pd.concat(arr_df)

In [ ]:
df_users.sample(5)

In [ ]:
df_users.info()

In [ ]:
df_users.reset_index(inplace=True)
df_users.drop("index", axis=1, inplace=True, errors="ignore")

In [ ]:
df_users.to_parquet(r"Generated\Yelp\users_extracted.parquet")

# NORMALIZACIÓN DE DATOS


### **GOOGLE**

#### _1. MERGE_SITE_REVIEWS_


In [8]:
df_maps_restaurantes = pd.read_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews = pd.read_parquet(r'Generated\Google\merge_site_reviews.parquet')
df_yelp_restaurantes = pd.read_parquet(r'Generated\Yelp\bussines.parquet')
df_yelp_checkin = pd.read_parquet(r'Generated\YELP\business_checkin.parquet')
df_yelp_tips = pd.read_parquet(r'Generated\Yelp\business_tip.parquet')
df_yelp_reviews = pd.read_parquet(r'Generated\Yelp\review.parquet')
df_yelp_users = pd.read_parquet(r'Generated\Yelp\users_extracted.parquet')

In [9]:
df_maps_reviews.sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,MISC,state,relative_results,url,state_ab,us_state,dtfmt,mes,dia,hora
1617111,105400992474227939958,Sikesrandy62 @Gmail Sikes,1568466452223,5,they got a lip snacking hot grits. eggs mouth ...,None,None,0x88dafd498b6d2845:0x96668ee40c965d73,2019,Florida,...,{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 7AM,"[0x88db02ac5728cd5f:0x89dd2ef880f38ea, 0x88daf...",https://www.google.com/maps/place//data=!4m2!3...,FL,Florida,2019-09-14 13:07:32.223,9,14,13
401867,111208830726888430553,P V,1513823884195,5,good food at this taco bell on a regular basis...,None,None,0x80dce86647c63e65:0x3714bd5d0e8413dd,2017,California,...,{'Accessibility': ['Wheelchair-accessible car ...,Open ⋅ Closes 2AM,"[0x80dce86552287869:0x99fdb6794cfcf8c3, 0x80dc...",https://www.google.com/maps/place//data=!4m2!3...,None,None,2017-12-21 02:38:04.195,12,21,2
1000317,115250175112809495709,Leomazz,1539001250446,5,(translated by google) sweetly fattening.\n\n(...,None,None,0x89c2599e515bf4dd:0x4f079e048a26869d,2018,New_York,...,{'Accessibility': ['Wheelchair-accessible entr...,Open 24 hours,"[0x89c259821bab0de1:0xc97c4df9856335f1, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...,None,None,2018-10-08 12:20:50.446,10,8,12


In [10]:
df_maps_reviews.columns.values

array(['user_id', 'name_x', 'time', 'rating', 'text', 'pics', 'resp',
       'gmap_id', 'anio', 'estado', 'name_y', 'address', 'description',
       'latitude', 'longitude', 'category', 'avg_rating',
       'num_of_reviews', 'price', 'hours', 'MISC', 'state',
       'relative_results', 'url', 'state_ab', 'us_state', 'dtfmt', 'mes',
       'dia', 'hora'], dtype=object)

In [11]:
df_maps_reviews.category

0                                        [Korean restaurant]
1                                        [Korean restaurant]
2                                        [Korean restaurant]
3                                        [Korean restaurant]
4                                        [Korean restaurant]
                                 ...                        
2393447    [Coffee shop, Bagel shop, Bakery, Breakfast re...
2393448    [Coffee shop, Bagel shop, Bakery, Breakfast re...
2393449    [Coffee shop, Bagel shop, Bakery, Breakfast re...
2393450    [Coffee shop, Bagel shop, Bakery, Breakfast re...
2393451    [Coffee shop, Bagel shop, Bakery, Breakfast re...
Name: category, Length: 2393452, dtype: object

In [12]:
df_maps_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393452 entries, 0 to 2393451
Data columns (total 30 columns):
 #   Column            Dtype         
---  ------            -----         
 0   user_id           object        
 1   name_x            object        
 2   time              int64         
 3   rating            int64         
 4   text              object        
 5   pics              object        
 6   resp              object        
 7   gmap_id           object        
 8   anio              int64         
 9   estado            object        
 10  name_y            object        
 11  address           object        
 12  description       object        
 13  latitude          float64       
 14  longitude         float64       
 15  category          object        
 16  avg_rating        float64       
 17  num_of_reviews    int64         
 18  price             object        
 19  hours             object        
 20  MISC              object        
 21  state   

In [13]:
df_maps_reviews[~df_maps_reviews.pics.isnull()].sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,MISC,state,relative_results,url,state_ab,us_state,dtfmt,mes,dia,hora
5002,116184702797654909865,Elizabeth Begonia,1514276702325,5,always great food and very authentic mexican f...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x80917ee02e1f1f3b:0xb26515c5c91119d4,2017,California,...,"{'Accessibility': None, 'Amenities': ['Good fo...",Permanently closed,"[0x80917ee17174c855:0xf49534a42f0176f1, 0x8091...",https://www.google.com/maps/place//data=!4m2!3...,CA,California,2017-12-26 08:25:02.325,12,26,8
37665,106853765567138662761,Monica Salas,1548972506086,5,great food !! had 8 different taco's and 1 ord...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x80853d94a3c0a79b:0xb8786eb9ae64068e,2019,California,...,"{'Accessibility': None, 'Amenities': None, 'At...",Temporarily closed,"[0x80853d540972c32d:0xb7c4c4e15bcebda5, 0x8085...",https://www.google.com/maps/place//data=!4m2!3...,CA,California,2019-01-31 22:08:26.086,1,31,22
2301739,115634094245909001959,James Bradford,1484533496131,1,total scumbags with disgusting food.\n\nordere...,[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89c6e7c2771e4b2f:0x1f07e995c0f7bb26,2017,Pennsylvania,...,{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 11PM,"[0x89c6e6510842b34d:0xa2ca08f5566e4cee, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...,PA,Pennsylvania,2017-01-16 02:24:56.131,1,16,2


In [14]:
num_nulls_pics = df_maps_reviews.pics.isnull().sum()
num_nulls_pics

2307136

In [15]:
num_nulls_resp = df_maps_reviews.resp.isnull().sum()
num_nulls_resp

2178044

In [16]:
df_maps_reviews["dtfmt"] = df_maps_reviews.time.apply(
    lambda x: datetime.utcfromtimestamp(x / 1000)
)

df_maps_reviews["mes"] = df_maps_reviews.dtfmt.dt.month

df_maps_reviews["dia"] = df_maps_reviews.dtfmt.dt.day

df_maps_reviews["hora"] = df_maps_reviews.dtfmt.dt.hour

df_maps_reviews.name_x = df_maps_reviews.name_x.str.title()

df_maps_reviews.text = df_maps_reviews.text.str.lower()

df_maps_reviews.sample(3)

,user_id,name_x,time,rating,text,pics,resp,gmap_id,anio,estado,...,MISC,state,relative_results,url,state_ab,us_state,dtfmt,mes,dia,hora
898475,114212667258716193560,Jose Baylon (Pillo),1493478405525,2,"04/29/17-they didn't have ketchup, no ice, had...",None,None,0x864ea68d5ac58e57:0x9839f69481344dbe,2017,Texas,...,{'Accessibility': ['Wheelchair accessible entr...,Open 24 hours,"[0x864ea6a02b5a5867:0x68a3cc634d173d92, 0x864e...",https://www.google.com/maps/place//data=!4m2!3...,TX,Texas,2017-04-29 15:06:45.525,4,29,15
595472,101861972990983849868,Richard S,1574091778780,3,food always seems luke warm.,None,None,0x864c2732089dcc1d:0xc22b80f939646ca9,2019,Texas,...,{'Accessibility': ['Wheelchair-accessible entr...,Closed ⋅ Opens 6:30AM,"[0x864c26d5b20dc6a7:0x93db2e43ca919555, 0x864c...",https://www.google.com/maps/place//data=!4m2!3...,None,None,2019-11-18 15:42:58.780,11,18,15
405788,104653875682646394410,Taylar Gantt,1506045042792,4,None,None,None,0x40771e59f0513085:0x513683476a2005d8,2017,California,...,{'Accessibility': ['Wheelchair-accessible entr...,Open ⋅ Closes 8PM,"[0x809b21d3b9ca90b9:0xe946282a6fd3d136, 0x809b...",https://www.google.com/maps/place//data=!4m2!3...,None,None,2017-09-22 01:50:42.792,9,22,1


#### **_<u>COMENTARIO</u>_**

Las columnas **pics** y **resp** deberían eliminarse porque la cantidades de datos nulos sobrepasa los 90%.

_Obtenemos las columnas de año, mes, día y hora para posteriores análisis_


# Salida Final ETL

In [17]:
df_maps_restaurantes.to_parquet(r'Generated\Google\metada_sitios.parquet')
df_maps_reviews.to_parquet(r'Generated\Google\merge_site_reviews.parquet')
df_yelp_restaurantes.to_parquet(r'Generated\Yelp\bussines.parquet')
df_yelp_checkin.to_parquet(r'Generated\YELP\business_checkin.parquet')
df_yelp_tips.to_parquet(r'Generated\Yelp\business_tip.parquet')
df_yelp_reviews.to_parquet(r'Generated\Yelp\review.parquet')
df_yelp_users.to_parquet(r'Generated\Yelp\users_extracted.parquet')